In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


# Import Library

In [18]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.3 MB 43.3 MB/s 
     |████████████████████████████████| 16.7 MB 418 kB/s 


In [25]:
import pandas as pd
import re
import geopandas as gpd
from shapely import wkt

# Read Data

In [3]:
pop_data = pd.read_csv('/content/drive/MyDrive/IE/WomanData/2021Census_G01_VIC_SAL.csv')
pop_data.head()

,SAL_CODE_2021,Tot_P_M,Tot_P_F,Tot_P_P,Age_0_4_yr_M,Age_0_4_yr_F,Age_0_4_yr_P,Age_5_14_yr_M,Age_5_14_yr_F,Age_5_14_yr_P,...,High_yr_schl_comp_Yr_8_belw_P,High_yr_schl_comp_D_n_g_sch_M,High_yr_schl_comp_D_n_g_sch_F,High_yr_schl_comp_D_n_g_sch_P,Count_psns_occ_priv_dwgs_M,Count_psns_occ_priv_dwgs_F,Count_psns_occ_priv_dwgs_P,Count_Persons_other_dwgs_M,Count_Persons_other_dwgs_F,Count_Persons_other_dwgs_P
0,SAL20001,4,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SAL20002,4637,4457,9088,168,171,340,195,204,397,...,162,31,37,64,4222,4094,8317,322,275,594
2,SAL20003,1929,2001,3925,91,60,144,256,241,491,...,97,8,12,22,1859,1922,3780,47,48,96
3,SAL20004,3,3,7,0,0,0,0,0,0,...,0,0,0,0,3,3,7,3,0,3
4,SAL20005,71,76,146,0,3,7,5,4,14,...,7,0,0,0,60,70,130,6,6,12


In [4]:
pop_meta_data = pd.read_excel('/content/drive/MyDrive/IE/WomanData/2021Census_geog_desc_1st_release.xlsx', sheet_name = '2021_ASGS_Non_ABS_Structures')
pop_meta_data.head()

,ASGS_Structure,Census_Code_2021,AGSS_Code_2021,Census_Name_2021,Area sqkm
0,AUS,AUS,AUS,AUSTRALIA,7.688095e+06
1,CED,CED101,101,Banks,4.943710e+01
2,CED,CED102,102,Barton,3.954700e+01
3,CED,CED103,103,Bennelong,5.869100e+01
4,CED,CED104,104,Berowra,7.439867e+02


# Cleaning

In [5]:
# select needed column
pop_data = pop_data.iloc[:, :4]
pop_data.head()

,SAL_CODE_2021,Tot_P_M,Tot_P_F,Tot_P_P
0,SAL20001,4,0,4
1,SAL20002,4637,4457,9088
2,SAL20003,1929,2001,3925
3,SAL20004,3,3,7
4,SAL20005,71,76,146


In [6]:
# check duplicate or missing
pop_data.isnull().any()

SAL_CODE_2021    False
Tot_P_M          False
Tot_P_F          False
Tot_P_P          False
dtype: bool

In [7]:
# check duplicated data
pop_data[pop_data.duplicated()]

,SAL_CODE_2021,Tot_P_M,Tot_P_F,Tot_P_P


In [8]:
# filter to SAL
pop_meta_data = pop_meta_data[pop_meta_data['ASGS_Structure'] == 'SAL']

## check missing or duplicated

In [9]:
pop_meta_data.isnull().any()

ASGS_Structure      False
Census_Code_2021    False
AGSS_Code_2021      False
Census_Name_2021    False
Area sqkm           False
dtype: bool

In [10]:
pop_meta_data.duplicated().any()

False

In [11]:
pop_meta_data = pop_meta_data.reset_index()
pop_meta_data.head()

,index,ASGS_Structure,Census_Code_2021,AGSS_Code_2021,Census_Name_2021,Area sqkm
0,3829,SAL,SAL10001,10001,Aarons Pass,82.7639
1,3830,SAL,SAL10002,10002,Abbotsbury,4.9788
2,3831,SAL,SAL10003,10003,Abbotsford (NSW),1.0180
3,3832,SAL,SAL10004,10004,Abercrombie,2.9775
4,3833,SAL,SAL10005,10005,Abercrombie River,127.1701


In [28]:
# convert to dictionary
SAL_dict = dict()
for i in range(len(pop_meta_data)):
  # pop_meta_data['Census_Name_2021'][i]
  SAL_dict[pop_meta_data['Census_Code_2021'][i]] = re.sub("[\(\[].*?[\)\]]", "", pop_meta_data['Census_Name_2021'][i]).strip()


In [29]:
# impute SAL name to pop_data
pop_data['Suburb'] = pop_data.apply(lambda x: SAL_dict[x['SAL_CODE_2021']], axis=1)
pop_data.head()

,SAL_CODE_2021,Tot_P_M,Tot_P_F,Tot_P_P,Suburb
0,SAL20001,4,0,4,Abbeyard
1,SAL20002,4637,4457,9088,Abbotsford
2,SAL20003,1929,2001,3925,Aberfeldie
3,SAL20004,3,3,7,Aberfeldy
4,SAL20005,71,76,146,Acheron


In [20]:
# read location data
df_loc = pd.read_csv('/content/drive/MyDrive/IE/CleanData/db_postcode_data.csv', sep=';')

# convert data type
df_loc['geometry'] = df_loc['geometry'].apply(wkt.loads)
df_loc = gpd.GeoDataFrame(df_loc, crs='epsg:4326')
df_loc['postcode'] = df_loc['postcode'].astype(int)

# read crime data
df_crime = pd.read_csv('/content/drive/MyDrive/IE/CleanData/db_crime_data.csv', sep=';')

# drop unnamed: 0 column
df_crime = df_crime.drop(['id'], axis =1)
df_crime.head()

,Year,Local Government Area,Offence Subdivision,Incidents Recorded,id_loc
0,2013,Alpine,Assault and related offences,5,355
1,2013,Alpine,Other crimes against the person,5,355
2,2013,Alpine,Dangerous and negligent acts endangering people,1,396
3,2013,Alpine,Assault and related offences,1,754
4,2013,Alpine,Dangerous and negligent acts endangering people,1,923


In [21]:
# join data
df = df_loc.join(df_crime.set_index('id_loc'), on='id')
df = df.drop(['id'], axis =1)

df[df['Year'].isnull()]
df['Year'].fillna(0, inplace=True)
df['Offence Subdivision'].fillna(0, inplace=True)
df['Incidents Recorded'].fillna(0, inplace=True)
df.head()

,LOC_NAME,geometry,postcode,latitude,longitude,Year,Local Government Area,Offence Subdivision,Incidents Recorded
0,Abbeyard,"POLYGON ((146.81721 -37.09735, 146.81729 -37.0...",3737,-37.021339,146.764079,2019.0,Alpine,Assault and related offences,2.0
1,Abbotsford,"POLYGON ((145.00235 -37.80723, 145.00350 -37.8...",3067,-37.804369,144.999700,2013.0,Yarra,Assault and related offences,36.0
1,Abbotsford,"POLYGON ((145.00235 -37.80723, 145.00350 -37.8...",3067,-37.804369,144.999700,2013.0,Yarra,Dangerous and negligent acts endangering people,2.0
1,Abbotsford,"POLYGON ((145.00235 -37.80723, 145.00350 -37.8...",3067,-37.804369,144.999700,2013.0,Yarra,Other crimes against the person,3.0
1,Abbotsford,"POLYGON ((145.00235 -37.80723, 145.00350 -37.8...",3067,-37.804369,144.999700,2013.0,Yarra,Robbery,7.0


# Calculate crime rate
dividing the number of reported crimes by the total population. The result is then multiplied by 100,000.

In [50]:
# convert to dictionary
# suburb_pop = dict()
# for i in range(len(pop_meta_data)):
#   # pop_meta_data['Census_Name_2021'][i]
#   SAL_dict[pop_meta_data['Census_Code_2021'][i]] = re.sub("[\(\[].*?[\)\]]", "", pop_meta_data['Census_Name_2021'][i]).strip
# int(pop_data[pop_data['Suburb'] == 'Stony Creek']['Tot_P_P'])
# pop_data[pop_data['Suburb'] == 'Stony Creek']['Tot_P_P']
pop_data[pop_data['Suburb'] == 'Stony Creek']

,SAL_CODE_2021,Tot_P_M,Tot_P_F,Tot_P_P,Suburb


In [51]:
df[df['LOC_NAME'] == 'Bridgewater on Loddon']

,LOC_NAME,geometry,postcode,latitude,longitude,Year,Local Government Area,Offence Subdivision,Incidents Recorded
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2013.0,Loddon,Assault and related offences,8.0
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2013.0,Loddon,Other crimes against the person,1.0
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2014.0,Loddon,Assault and related offences,6.0
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2014.0,Loddon,Dangerous and negligent acts endangering people,1.0
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2014.0,Loddon,"Stalking, harassment and threatening behaviour",1.0
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2015.0,Loddon,Assault and related offences,2.0
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2015.0,Loddon,Other crimes against the person,1.0
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2016.0,Loddon,Assault and related offences,2.0
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2017.0,Loddon,Assault and related offences,4.0
353,Bridgewater on Loddon,"POLYGON ((143.93445 -36.60764, 143.93460 -36.6...",3516,-36.603609,143.94076,2017.0,Loddon,"Stalking, harassment and threatening behaviour",1.0


In [49]:
zero_sub = pop_data[pop_data['Tot_P_P'] == 0]['Suburb'].to_list()
pop_sub =  pop_data['Suburb'].to_list()
subs = df['LOC_NAME'].to_list()
for i in subs:
  if i not in pop_sub:
    print(i)

Arapiles
Banksia Peninsula
Baynton East
Bend of Islands
Bend of Islands
Bend of Islands
Bend of Islands
Bend of Islands
Blackheath
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Bridgewater on Loddon
Broadlands
Condah Swamp
Coopers Creek
Fieldstone
Granite Flat
Grass Flat
Inkerman
Jilpanger
Jung
Larralea
Lyal
Mount Eccles South
Moyarra
Murray-Sunset
Queensferry
Shannonvale
Speewa
Stanhope South
Tantaraboo
Thalloo
Tidal River
Tylden South
Wanalta
Waranga
Woodside North
Woodstock on Loddon
Woodstock on Loddon


In [52]:
# def calculate_rate(records, suburb):
#   if suburb in pop_data['Suburb'].to_list():
#     if pop_data[pop_data['Suburb'] == suburb]['Tot_P_P'] == 0:
#       return -1
#       return records * 100000 / pop_data[pop_data['Suburb'] == suburb]['Tot_P_P']+1
#   else:
#     return -1

In [53]:
# join data with population
df['crime_rate'] = df.apply(lambda x: x['Incidents Recorded'] * 100000 / max(pop_data[pop_data['Suburb'] == x['LOC_NAME']]['Tot_P_P']+1) if x['LOC_NAME'] in pop_data['Suburb'].to_list() else -1 , axis=1)
# df['crime_rate'] = df.apply(calculate_rate())
df.head()

,LOC_NAME,geometry,postcode,latitude,longitude,Year,Local Government Area,Offence Subdivision,Incidents Recorded,crime_rate
0,Abbeyard,"POLYGON ((146.81721 -37.09735, 146.81729 -37.0...",3737,-37.021339,146.764079,2019.0,Alpine,Assault and related offences,2.0,40000.000000
1,Abbotsford,"POLYGON ((145.00235 -37.80723, 145.00350 -37.8...",3067,-37.804369,144.999700,2013.0,Yarra,Assault and related offences,36.0,396.083177
1,Abbotsford,"POLYGON ((145.00235 -37.80723, 145.00350 -37.8...",3067,-37.804369,144.999700,2013.0,Yarra,Dangerous and negligent acts endangering people,2.0,22.004621
1,Abbotsford,"POLYGON ((145.00235 -37.80723, 145.00350 -37.8...",3067,-37.804369,144.999700,2013.0,Yarra,Other crimes against the person,3.0,33.006931
1,Abbotsford,"POLYGON ((145.00235 -37.80723, 145.00350 -37.8...",3067,-37.804369,144.999700,2013.0,Yarra,Robbery,7.0,77.016173


# export csv

In [55]:
df.to_csv('/content/drive/MyDrive/IE/WomanData/df_crime_rate.csv')